Library Insatallation

In [ ]:
!pip install kaggle
!pip install datasets

In [ ]:
!pip install -U unsloth unsloth-zoo transformers peft accelerate --quiet
!pip install trl==0.23.0 --quiet

In [ ]:
!pip install evaluate
!pip install rouge_score
!pip install bert_score

In [ ]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d fatmaalzhraahmed/shifaa-arabic-medical-q-and-a

Dataset URL: https://www.kaggle.com/datasets/fatmaalzhraahmed/shifaa-arabic-medical-q-and-a
License(s): apache-2.0
  0% 0.00/78.4M [00:00<?, ?B/s]
100% 78.4M/78.4M [00:00<00:00, 1.01GB/s]


In [ ]:
!unzip shifaa-arabic-medical-q-and-a.zip

Archive:  shifaa-arabic-medical-q-and-a.zip
  inflating: Shifaa Arabic Medical Q and A.csv  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.model_selection import train_test_split
from datasets import load_dataset,DatasetDict
from transformers import pipeline
import random
import pandas as pd
import statistics
import evaluate
from IPython.display import display, Markdown

In [ ]:
df=pd.read_csv("/content/shifaa-arabic-medical-q-and-a.zip")
df

,Question,Answer
0,أعاني من ألم فوق وبجانب السرة وكأنه طعن بسكين ...,منى إن الأدوية التي تستعملينها (ميتوتركسات وال...
1,عمري 31 سنة ومتزوجة منذ 8 أشهر ولم يحصل الحمل ...,سارة أتفهم مشاعرك ولهفتك على الحمل يا عزيزتي ف...
2,سؤالي عندي طفل يبلغ مع العمر 6 سنوات المشكلة ه...,قد يتسبب الإمساك المزمن عند الطفل بسبب نقص الس...
3,تعرضت لحادث في المباراة وعملت صورة رنين مغناطي...,حبذا لو ذكرت لنا كم مضى على تمزق الرباط وهل هو...
4,أنا شاب عمري 22 عاما أعاني من القولون العصبي م...,لا يمكن لمن يتناول الحلويات والسكريات والمشروب...
...,...,...
84417,أريد أن أعرف ما هي أسباب اسمرار مفاصل اليد حتى...,سميرة المناطق المعرضة للاحتكاك بالمحيط الخارجي...
84418,أعاني من كثافة الشعر على الساعدين والرجلين بشك...,فإن الاختلاف بين الناس وخلقهم وطبائعهم هو أمر ...
84419,ما هو مرض الزلال فقد قمت بفحص البول وأخبرني ال...,الأخ الفاضل أبو زهور الزلال كلمة تطلق على ظهور...
84420,أنا فريال من الجزائر من فضلكم أريد أن أعرف متى...,فريال وبعد عوضك الله بكل خير وجعل صبرك واحتساب...


In [ ]:
df.dropna(subset='Question', inplace=True)

In [ ]:
data_for_model = []
for index, row in df.iterrows():
    data_for_model.append({
        "instruction": row['Question'],
        "output": row['Answer']
    })

In [ ]:
with open('data_ready_for_model.jsonl', 'w', encoding='utf-8') as f:
    for i in data_for_model:
        f.write(json.dumps(i, ensure_ascii=False) + "\n")

In [ ]:
full_dataset = load_dataset("json", data_files="/content/data_ready_for_model.jsonl", split='train')

Generating train split: 0 examples [00:00, ? examples/s]

# Split Data Into Train, Test, Val
### With size :


1.   Train : 80% (**67535**)
2.   Validation: 15% (**12663**)
3.   Test :5%  (**4221**)



In [ ]:
train_test_split = full_dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
validation_test_split = train_test_split['test'].train_test_split(test_size=0.25, seed=42)

In [ ]:
final_dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': validation_test_split['train'],
    'test': validation_test_split['test']
})

In [ ]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 67535
    })
    validation: Dataset({
        features: ['instruction', 'output'],
        num_rows: 12663
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 4221
    })
})

In [ ]:
train_dataset =  final_dataset['train']
val_dataset = final_dataset['validation']
test_dataset = final_dataset['test']

In [ ]:
train_dataset[6]

{'instruction': 'منذ 7 أشهر أجريت عمليه تغيير الصمام الرئوي جراحة في أمريكا والحمد لله بعد العملية لم يكن هناك أي ألم ولكن الآن مع أي مجهود أشعر بألم في قلبي وصدري وصعوبة في التنفس وخفقان هل هو من القلب أم ماذا',
 'output': 'وردة وبعد فلم تذكري نوع العملية التي تم إجراؤها فهل تم تغيير الصمام عن طريق القسطرة أم عن طريق فتح الصدر على كل حال لابد وأنه قد تم شرح مضاعفات العملية لك فأحيانا يحصل قصور في الصمام وأحيانا قد لا يتحسن التضييق في الصمام وأحيانا قد يكون هناك مضاعفات أخرى ولذا ففي مثل حالتك يجب مراجعة الطبيب المتابع لحالتك في مستشفى حمد وبسرعة ليتم معرفة سبب آلام الصدر وضيق النفس والخفقان خاصة إن كانت هذه الأعراض قد تحسنت بعد العملية كما ذكرت في رسالتك الطبيب المعالج العارف لحالتك هو من يستطيع بعد الفحص الطبي وبعد إجراء بعض الصور والإجراءات التشخيصية أن يقدر وضع الصمام وسبب الأعراض الجديدة والله الموفق'}

In [ ]:
from huggingface_hub import login
token = ""
login(token)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,AutoModelForSequenceClassification
from unsloth import FastLanguageModel

/tmp/ipython-input-4100157713.py:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
import torch
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

In [ ]:
path = "Fatma5000/llama_arabic_medical"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=path,
    max_seq_length=1024,
    token=token,
    quantization_config=bnb_config,
    dtype = None,
    load_in_4bit = True,
)


pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

Unsloth 2025.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
Device set to use cuda:0


In [ ]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [ ]:
prompt_template = """<|start_header_id|>system<|end_header_id|>
أنت "شِفاء"، مساعد طبي خبير وموثوق. مهمتك هي تحليل الاستفسارات الطبية وتقديم إجابات واضحة، منظمة، ومتعاطفة لمساعدة المستخدمين على فهم حالتهم الصحية بشكل أفضل. استخدم دائمًا تنسيق Markdown (عناوين ونقاط) لتسهيل القراءة.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""


In [ ]:
my_question = "ما هي أسباب ألم أسفل الظهر عند الرجال؟"
prompt = prompt_template.format(my_question)
outputs = pipe(prompt, max_new_tokens=500)
print(outputs[0]['generated_text'])

<|start_header_id|>system<|end_header_id|>
أنت "شِفاء"، مساعد طبي خبير وموثوق. مهمتك هي تحليل الاستفسارات الطبية وتقديم إجابات واضحة، منظمة، ومتعاطفة لمساعدة المستخدمين على فهم حالتهم الصحية بشكل أفضل. استخدم دائمًا تنسيق Markdown (عناوين ونقاط) لتسهيل القراءة.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
ما هي أسباب ألم أسفل الظهر عند الرجال؟<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
شكرا على تواصلك مع الشبكة الإسلامية إن أسباب ألم الظهر متعددة وغالبا ما يكون سبب الألم هو العضلات والأربطة في الظهر إلا أن أسباب أخرى كثيرة تؤدي إلى آلام الظهر ومن هذه الأسباب 1 ديسك الظهر وهو انزلاق غضروفي يؤدي إلى ضغط على جذر العصب مما يؤدي إلى الألم والتنميل في الطرف السفلي أو في الطرفين في بعض الحالات ويزداد الألم مع الحركة ومع السعال والعطاس أو الانحناء إلى الأمام في وضعية القرفصاء أو الجلوس الطويل 2 الانزلاق الغضروفي بين الفقرات ويسمى بروزات عظمية وهي تسبب أعراضا مشابهة لانسداد القناة الشوكية ويكون العلاج في هذه الحالة إما بالعلاج المحافظ بالمسكنات والمتابعة الطبية أو بالتدخل الج

In [ ]:
my_question=test_dataset[0]['instruction']
prompt = prompt_template.format(my_question)
outputs=pipe(prompt, max_new_tokens=500)
Markdown(outputs[0]['generated_text'])

<|start_header_id|>system<|end_header_id|>
أنت "شِفاء"، مساعد طبي خبير وموثوق. مهمتك هي تحليل الاستفسارات الطبية وتقديم إجابات واضحة، منظمة، ومتعاطفة لمساعدة المستخدمين على فهم حالتهم الصحية بشكل أفضل. استخدم دائمًا تنسيق Markdown (عناوين ونقاط) لتسهيل القراءة.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
أنا فتاة غير متزوجة عمري 23 سنة أعاني منذ 5 أو 4 سنوات عدم انتظام الدورة إما تكون منقطعة لثلاثة أو أربعة شهور أو تأتي لمدة 17 أو 18 يوما وقد تصل إلى 20 يوما وتنقطع لمدة أسبوع أو أسبوع ونصف وتعود مرة أخرى وهذه الحالات كانت تأتيني في بداية الأمر أما في السنتين الأخيرتين فقد أصبحت تنقطع لفترة من شهرين إلى ثلاثة وتعود على شكل نقاط وتنقطع أو تأتي بشكل قليل لفترة ثم تتحول قطرات والعكس هذا الأمر أصبح يؤرقني خاصة في الأمور الدينية حيث إنه في بعض المرات يلتبس علي الأمر هل هو حيض أم استحاضة مع العلم أن الدم أحيانا يكون ذا رائحة وغزير ومعه بعض القطع ويستمر على هذا الحال بنفس الوصف 18 يوما مع العلم أنه لا تأتيني آلام قوية وإنما ألم خفيف في البطن والظهر ويختفي بسرعة عندما تنزل وأحيانا لا يأتيني أبدا وهو الغالب لكن نفسيتي تتأثر قبل نزول الدم حيث إني أصبح حساسة جدا ولا أتحمل الضوضاء وأغضب كثيرا وغالبا حتى عند نزول القطرات ألاحظ تقلب حالتي النفسية مع العلم أني قد ذهبت لطبيبة النساء والولادة وقد عملت تحاليل عند بداية الأمر وأشعة صوتية لمنطقة الرحم ولم يكن هناك سبب واضح لحالتي بل أن كل شيء سليم وصرفت لي الطبيبة منظم هرمونات واستخدمته لفترة وانتظمت دورتي لفترة ثم عادت وذهبت مرة أخرى وصرفت لي الدواء ونفس النتيجة تنتظم لفترة ثم تعود أصبت في السنة الثانية بأنيميا مع نقص في الهيموجلويين وأصبح تركيزي ضعيفا وكذلك ذاكرتي وأصبحت أعاني من التذكر والسرحان في بعض الأحيان وزاد وزني وأصبحت كسولة جدا وأنام كثيرا وأشعر بالإرهاق عند أدنى مجهود ولا أعلم لماذا دورتي لا تتنظم أنا فتاة في مقتبل العمر وأخشى أن تؤثر على حياتي الزوجية مستقبلا أو تؤثر على الإنجاب فما السبب هل ممكن أن يكون السبب نفسيا وما الحل أرجو مساعدتي وجزاكم الله خيرا<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
نور حسب ما تبين لي من رسالتك فإن دورتك الشهرية غير منتظمة وهذا يؤكد على وجود خلل هرموني ما في جسمك وهذا الخلل قد يكون بسيطا جدا لكنه قد يكون شديدا أيضا لذلك فيجب عمل بعض التحاليل الهرمونية الأساسية وذلك للتأكد من عدم وجود اضطراب في عمل الغدة الدرقية أو الكظرية أو النخامية أو ارتفاع في هرمون الحليب أو تكيس في المبايض أو غير ذلك والتحاليل هي LHFSHTOTAL AND FREE TESTOSTERONTSHFREE T3T4PROLACTINDHEAS ويجب عملها في ثاني أو ثالث يوم من الدورة وفي الصباح فإن تبين وجود سبب مثل ارتفاع هرمون الحليب أو اضطراب الغدة الدرقية أو تكيس المبايض أو غير ذلك فبعلاج السبب ستعود الدورة منتظمة بإذن الله تعالى أما إن كانت التحاليل طبيعية فالحالة ستكون ناتجة عن وجود حالة تكيس على المبايض لكنها غير ظاهرة بالتحاليل ويمكن علاجها بنفس الطريقة التي يتم فيها علاج تكيس المبايض ويمكنك تناول حبوب تسمى (غلكوفاج) حبة واحدة يوميا من عيار 500 ملغ في الأسبوع الأول ثم حبتين في الأسبوع الثاني ثم ثلاث حبات في الأسبوع الثالث ثم استمر على ثلاث حبات بعد ذلك وإنقاص الوزن من خلال حمية غذائية جيدة وممارسة الرياضة مع ضرورة تناول حبوب منع الحمل من النوع الثنائي الهرمون مثل حبوب (ياسمين) فهذه الحبوب ستعالج حالة التكيس وستحمي المبيض من أي تليف أو أورام في المستقبل بإذن الله تعالى لذلك أنصحك بتناول حبوب منع الحمل من هذا النوع لمدة لا تقل عن 6 أشهر ثم التوقف عنها ويمكنك تكرارها بعد ذلك لمدة 6 أشهر أخرى ثم التوقف عنها كليا ويمكنك تناول حبوب (دوفاستون) حبتين يوميا من ثاني يوم من نزول الدورة

In [ ]:
my_question =test_dataset[2]['instruction']
prompt = prompt_template.format(my_question)
outputs = pipe(prompt, max_new_tokens=600)
print(test_dataset[2]['instruction'],"\n")
full_text = outputs[0]['generated_text']
answer = full_text.split("<|start_header_id|>assistant<|end_header_id|>")[-1].strip()
answer = answer.replace("<|eot_id|>", "").strip()

Markdown(answer)

In [ ]:
answer_data=test_dataset[2]['output']
Markdown(answer_data)

In [ ]:
import evaluate
from tqdm import tqdm

my_question = test_dataset[12]['instruction']
prompt = prompt_template.format(my_question)
outputs = pipe(prompt, max_new_tokens=600,temperature=0.2)


Markdown( test_dataset[12]['instruction'])

In [ ]:
answer_reference = test_dataset[12]['output']
display(Markdown(answer_reference))
print("\n" + "="*60 + "\n")

In [ ]:
full_text = outputs[0]['generated_text']
answer_generated = full_text.split("<|start_header_id|>assistant<|end_header_id|>")[-1].strip()
answer_generated = answer_generated.replace("<|eot_id|>", "").strip()

display(Markdown(answer_generated))
print("\n" + "="*60 + "\n")

In [ ]:
predictions = [answer_generated.strip()]
references = [answer_reference.strip()]

In [ ]:
exact_match = 1 if predictions[0].lower() == references[0].lower() else 0
print("📊 Exact Match:", exact_match * 100, "%")

📊 Exact Match: 0 %


In [ ]:
# ========== 2. ROUGE-L ==========
print("\n📊 Calculating ROUGE-L...")
rouge = evaluate.load("rouge")
rouge_results = rouge.compute(predictions=predictions, references=references)
print("🔹 ROUGE-L Score:", round(rouge_results["rougeL"] * 100, 2), "%")

# ========== 3. BLEU ==========
print("\n📊 Calculating BLEU...")
bleu = evaluate.load("bleu")
bleu_results = bleu.compute(predictions=predictions, references=references)
print("🔹 BLEU Score:", round(bleu_results["bleu"] * 100, 2), "%")

# ========== 4. BERTScore ==========
print("\n🔍 Calculating BERTScore...")
bertscore = evaluate.load("bertscore")
bertscore_results = bertscore.compute(
    predictions=predictions,
    references=references,
    lang="ar"
)
bertscore_precision = bertscore_results["precision"][0]
bertscore_recall = bertscore_results["recall"][0]
bertscore_f1 = bertscore_results["f1"][0]

print("\n🔹 BERTScore:")
print("   • Precision:", round(bertscore_precision * 100, 2), "%")
print("   • Recall:   ", round(bertscore_recall * 100, 2), "%")
print("   • F1:       ", round(bertscore_f1 * 100, 2), "%")


📊 Calculating ROUGE-L...


🔹 ROUGE-L Score: 0.0 %

📊 Calculating BLEU...


🔹 BLEU Score: 0.8 %

🔍 Calculating BERTScore...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


🔹 BERTScore:
   • Precision: 69.3 %
   • Recall:    68.46 %
   • F1:        68.88 %


# **Antcoloy Algo**

In [ ]:
class LLMEvaluator:
    def __init__(self, pipe, dataset, prompt_template):
        self.pipe = pipe
        self.dataset = dataset
        self.prompt_template = prompt_template
        print("Loading metrics (BERTScore)...")
        self.bertscore = evaluate.load("bertscore")

    def get_fitness_score(self, params, sample_indices=[10, 11, 12]):
        scores = []
        last_generated_text = ""

        for idx in sample_indices:
            instruction = self.dataset[idx]['instruction']
            reference = self.dataset[idx]['output']
            prompt = self.prompt_template.format(instruction, "")

            temp = max(0.01, params['temperature'])
            top_p = params['top_p']
            max_tokens = int(params['max_tokens'])

            try:
                outputs = self.pipe(
                    prompt,
                    max_new_tokens=max_tokens,
                    temperature=temp,
                    top_p=top_p,
                    do_sample=True
                )
                full_text = outputs[0]['generated_text']

                generated_text = full_text.split("<|start_header_id|>assistant<|end_header_id|>")[-1].strip()
                generated_text = generated_text.replace("<|eot_id|>", "").strip()

                last_generated_text = generated_text

                results = self.bertscore.compute(
                    predictions=[generated_text],
                    references=[reference],
                    lang="ar"
                )
                scores.append(results["f1"][0])

            except Exception as e:
                print(f"Error during generation: {e}")
                scores.append(0.0)


        if not scores:
            return 0.0, 0.0, 0.0, ""

        ant_max = max(scores)
        ant_mean = statistics.mean(scores)
        ant_median = statistics.median(scores)

        return ant_mean, ant_max, ant_median, last_generated_text



In [ ]:
class AntColonyOptimizer:
    def __init__(self, evaluator, search_space, config):
        self.evaluator = evaluator
        self.search_space = search_space
        self.num_ants = config.get('num_ants', 5)
        self.num_iterations = config.get('iterations', 3)
        self.evaporation_rate = config.get('evaporation_rate', 0.5)
        self.alpha = config.get('alpha', 2.0)
        self.pheromones = {}
        for param_name, values in search_space.items():
            self.pheromones[param_name] = {v: 1.0 for v in values}

        self.best_global_score = -1
        self.best_params = {}
        self.random_seed = config.get('random_seed', 42)

    def _select_value(self, param_name):
        possible_values = list(self.pheromones[param_name].keys())
        pheromone_values = list(self.pheromones[param_name].values())
        weights = [p ** self.alpha for p in pheromone_values]
        return random.choices(possible_values, weights=weights, k=1)[0]

    def _update_pheromones(self, iteration_results):
        for param in self.pheromones:
            for value in self.pheromones[param]:
                self.pheromones[param][value] *= (1 - self.evaporation_rate)

        for result in iteration_results:
            score = result['score']
            params = result['params']
            for key, value in params.items():
                self.pheromones[key][value] += score

    def display_pheromone_status(self, iteration):
        print(f"\n --- Pheromone Status (Iteration {iteration}) ---")
        for param_name in self.pheromones:
            df = pd.DataFrame(list(self.pheromones[param_name].items()), columns=['Value', 'Pheromone'])
            df = df.sort_values(by='Pheromone', ascending=False).head(3)
            print(f"🔹 {param_name}:\n{df.to_string(index=False)}\n")
        print("-" * 40)

    def optimize(self):
        print("Starting Optimization Process (Optimization Target: Median)...")
        random.seed(self.random_seed)

        num_examples = 10
        dataset_size = len(self.evaluator.dataset)

        test_samples = random.sample(range(dataset_size), num_examples)

        print(f"Selected Test Indices: {test_samples}")

        for i in range(1, self.num_iterations + 1):
            self.display_pheromone_status(i)
            iteration_results = []

            for ant_id in range(self.num_ants):
                current_params = {
                    key: self._select_value(key) for key in self.search_space
                }

                mean_score, max_score, median_score, _ = self.evaluator.get_fitness_score(current_params, test_samples)

                iteration_results.append({
                    'params': current_params,
                    'score': median_score
                })

                print(f"   🐜 Ant {ant_id+1}: {current_params}")
                print(f"       📊 Stats -> Mean: {mean_score:.4f} | Max: {max_score:.4f} | Median: {median_score:.4f}")

                if median_score > self.best_global_score:
                    self.best_global_score = median_score
                    self.best_params = current_params
                    print(f"       🌟 New Best Found (by Median)! ({median_score:.4f})")

                print("   " + "-"*30)

            self._update_pheromones(iteration_results)

        print("\n Optimization Finished!")
        print("\n Generating Final Answer with BEST parameters...")

        _, _, final_median, final_text = evaluator.get_fitness_score(best_params)

        display(Markdown(f"**Best Generated Answer (Median Score: {final_median:.4f}):**\n\n{final_text}"))
        print(f" Best Parameters: {self.best_params}")

        return self.best_params

In [ ]:
import random
import pandas as pd
search_space = {
    'temperature': [0.1,0.2,0.4, 0.3, 0.5, 0.7, 0.9, 1.0],
    'top_p':       [0.5, 0.7, 0.9, 0.95, 0.99],
    'max_tokens':  [128, 256, 512, 1024]
}
aco_config = {
    'num_ants': 5,
    'iterations': 4,
    'evaporation_rate': 0.7,
    'alpha': 2.0
}
evaluator = LLMEvaluator(pipe, test_dataset, prompt_template)
optimizer = AntColonyOptimizer(evaluator, search_space, aco_config)
best_params = optimizer.optimize()
print("\n Generating Final Answer with BEST parameters...")
_, _, final_score, final_text = evaluator.get_fitness_score(best_params)
display(Markdown(f"**Best Generated Answer:**\n\n{final_text}"))

Loading metrics (BERTScore)...
Starting Optimization Process (Optimization Target: Median)...
Selected Test Indices: [912, 204, 2253, 2006, 1828, 1143, 839, 712, 3456, 260]

 --- Pheromone Status (Iteration 1) ---
🔹 temperature:
 Value  Pheromone
   0.1        1.0
   0.2        1.0
   0.4        1.0

🔹 top_p:
 Value  Pheromone
   0.5        1.0
   0.7        1.0
   0.9        1.0

🔹 max_tokens:
 Value  Pheromone
   128        1.0
   256        1.0
   512        1.0

----------------------------------------
   🐜 Ant 1: {'temperature': 0.1, 'top_p': 0.7, 'max_tokens': 512}
       📊 Stats -> Mean: 0.6817 | Max: 0.7171 | Median: 0.6946
       🌟 New Best Found (by Median)! (0.6946)
   ------------------------------
   🐜 Ant 2: {'temperature': 0.1, 'top_p': 0.5, 'max_tokens': 512}
       📊 Stats -> Mean: 0.6805 | Max: 0.7435 | Median: 0.6954
       🌟 New Best Found (by Median)! (0.6954)
   ------------------------------
   🐜 Ant 3: {'temperature': 0.5, 'top_p': 0.7, 'max_tokens': 512}
      

**Best Generated Answer (Median Score: 0.6415):**

بالطبع يستمر العلاج لحين الانتهاء من الحمل والرضاعة لذلك يمكن إكمال العلاج مع المتابعة المستمرة للزوجة مع الطبيبة المعالجة وللتأكد من سلامة الجنين مع إجراء التصوير التلفزيوني المنتظم وقياس حيوية وقياسات عضلة القلب والاطمئنان على نمو وحركة الجنين بشكل طبيعي والله الموفق

 Best Parameters: {'temperature': 0.7, 'top_p': 0.7, 'max_tokens': 512}

 Generating Final Answer with BEST parameters...


**Best Generated Answer:**

فإن (Anti phospholipid syndrome) أو متلازمة (الـ antiphospholipid syndrome) هي متلازمة أو مجموعة من الأعراض التي تنتج عن وجود أجسام مضادة في الدم تسمى (antiphospholipid antibodies) وهي تؤدي إلى ظهور أعراض في الجسم مثل ظهور جلطات في الأطراف أو في المشيمة أو حدوث إجهاض متكرر أو ولادة مبكرة أو حدوث أعراض أخرى مثل الصداع أو آلام المفاصل أو آلام الصدر أو غير ذلك وقد تؤثر هذه الأجسام المضادة على أوعية الجسم الرئيسية مثل أوعية القلب أو الرئتين أو الكليتين أو المخ أو أوعية المشيمة في الحمل وقد تؤدي إلى جلطات في هذه الأوعية أو إلى انسدادها أو إلى حدوث إجهاض أو ولادة مبكرة أو موت الجنين في البطن بعد الولادة أو غير ذلك من الأعراض التي تؤثر على صحة الأم والجنين وبالتالي فإن متلازمة (antiphospholipid syndrome) تعتبر حالة طبية هامة يجب علاجها بشكل جيد حتى لا تؤثر على صحة الأم والجنين إن شاء الله تعالى ويمكن تشخيص هذه الحالة عن طريق عمل بعض الفحوصات الطبية مثل (Anti phospholipid antibodies Anti cardiolipin antibodies Lupus anticoagulant) وغير ذلك ويمكن إعطاء حبوب (الأسبرين) مع حبوب (الهيبارين) لمدة ستة أشهر على الأقل ثم إعادة تقييم الحالة بعد ذلك ويمكن إعطاء حبوب (الهيبارين) فقط إذا كان لدى السيدة إجهاضات أو ولادة مبكرة سابقا ويمكن إعطاء حبوب (الأسبرين) فقط إذا لم يكن لدى السيدة إجهاضات أو ولادة مبكرة سابقا ويمكن إعطاء حبوب (الأسبرين) مع حبوب (البنادول) إذا كانت السيدة تعاني من صداع أو آلام في المفاصل أو غير ذلك ويمكن إعطاء حبوب (الأسبرين) مع حبوب (البريدنيزون) إذا كان لدى السيدة إجهاضات أو ول